# Using  Evidently to Evaluate Data Drift for Dataset

This notebook shows how you can use Evidently to check the data for data drift.

Acknowledgments:

The dataset used in the example is from: https://www.kaggle.com/c/bike-sharing-demand/data?select=train.csv
Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg
More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

## Getting Started¶
To run this tutorial:

1. Install MLflow
You can install MLflow with the following command `pip install mlflow` or install MLflow with scikit-learn via `pip install mlflow[extras]`
More details:https://mlflow.org/docs/latest/tutorials-and-examples/tutorial.html#id5

2. Install Evidently
You can install Evidently with the following command `pip install evidently`
In case you are also interested in Evidently Dashboard visualization in Jupyter install jupyter nbextention:
`jupyter nbextension install --sys-prefix --symlink --overwrite --py evidently`
And activate it:
`jupyter nbextension enable evidently --py --sys-prefix`
More details: https://docs.evidentlyai.com/install-evidently 

3. Optionally, you can load data from https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset and save in locally or skip this step and download data with  ```requests```  using instructions below

In [ ]:
import json
import pandas as pd
import numpy as np
import requests
import zipfile
import io

import plotly.offline as py #working offline
import plotly.graph_objs as go

from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
py.init_notebook_mode()

## Load Data

In [ ]:
content = requests.get("https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip").content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("day.csv"), header=0, sep=',', parse_dates=['dteday'])

In [ ]:
#observe data structure
raw_data.head()

In [ ]:
#set column mapping for Evidently Profile
data_columns = ColumnMapping()
data_columns.datetime = 'dteday'
data_columns.numerical_features = ['weathersit', 'temp', 'atemp', 'hum', 'windspeed']

In [ ]:
#set reference dates
reference_dates = ('2011-01-01 00:00:00','2011-01-28 23:00:00')

#set experiment batches dates
experiment_batches = [
    ('2011-02-01 00:00:00','2011-02-28 23:00:00'),
    ('2011-03-01 00:00:00','2011-03-31 23:00:00'),
    ('2011-04-01 00:00:00','2011-04-30 23:00:00'),
    ('2011-05-01 00:00:00','2011-05-31 23:00:00'),  
    ('2011-06-01 00:00:00','2011-06-30 23:00:00'), 
    ('2011-07-01 00:00:00','2011-07-31 23:00:00'), 
]

## Functions to calculate drift with Evidently

In [ ]:
data_drift_report = Report(metrics=[DataDriftPreset()])
data_drift_report.run(reference_data=raw_data[:100], current_data=raw_data[100:200], column_mapping=data_columns)
report = data_drift_report.as_dict()

In [ ]:
report["metrics"][1]["result"]["drift_by_columns"]

In [ ]:
#evaluate data drift with Evidently Profile
def detect_dataset_drift(reference, production, column_mapping, get_ratio=False):
    """
    Returns True if Data Drift is detected, else returns False.
    If get_ratio is True, returns the share of drifted features.
    The Data Drift detection depends on the confidence level and the threshold.
    For each individual feature Data Drift is detected with the selected confidence (default value is 0.95).
    Data Drift for the dataset is detected if share of the drifted features is above the selected threshold (default value is 0.5).
    """
    
    data_drift_report = Report(metrics=[DataDriftPreset()])
    data_drift_report.run(reference_data=reference, current_data=production, column_mapping=column_mapping)
    report = data_drift_report.as_dict()
    
    if get_ratio:
        return report["metrics"][0]["result"]["drift_share"]
    else:
        return report["metrics"][0]["result"]["dataset_drift"]


In [ ]:
#evaluate data drift with Evidently Profile
def detect_features_drift(reference, production, column_mapping, get_scores=False):
    """
    Returns True if Data Drift is detected, else returns False. 
    If get_scores is True, returns scores value (like p-value) for each feature.
    The Data Drift detection depends on the confidence level and the threshold.
    For each individual feature Data Drift is detected with the selected confidence (default value is 0.95).
    """
    
    data_drift_report = Report(metrics=[DataDriftPreset()])
    data_drift_report.run(reference_data=reference, current_data=production, column_mapping=column_mapping)
    report = data_drift_report.as_dict()
    
    drifts = []
    num_features = column_mapping.numerical_features if column_mapping.numerical_features else []
    cat_features = column_mapping.categorical_features if column_mapping.categorical_features else []
    for feature in num_features + cat_features:
        drift_score = report["metrics"][1]["result"]["drift_by_columns"][feature]["drift_score"]
        if get_scores:
            drifts.append((feature, drift_score))
        else:
            drifts.append((feature, report["metrics"][1]["result"]["drift_by_columns"][feature]["drift_detected"]))
             
    return drifts

## Features Drift

In [ ]:
features_historical_drift = []

for date in experiment_batches:
    drifts = detect_features_drift(raw_data.loc[raw_data.dteday.between(reference_dates[0],reference_dates[1])], 
                           raw_data.loc[raw_data.dteday.between(date[0], date[1])], 
                           column_mapping=data_columns)
    
    features_historical_drift.append([x[1] for x in drifts])
    
features_historical_drift_frame = pd.DataFrame(features_historical_drift, 
                                               columns = data_columns.numerical_features)

In [ ]:
fig = go.Figure(data=go.Heatmap(
                   z = features_historical_drift_frame.astype(int).transpose(),
                   x = [x[1] for x in experiment_batches],
                   y = data_columns.numerical_features,
                   hoverongaps = False,
                   xgap = 1,
                   ygap = 1,
                   zmin = 0,
                   zmax = 1,
                   showscale = False,
                   colorscale = 'Bluered'
))

fig.update_xaxes(side="top")

fig.update_layout(
    xaxis_title = "Timestamp",
    yaxis_title = "Feature Drift"
)

fig.show()

In [ ]:
features_historical_drift_pvalues = []

for date in experiment_batches:
    drifts = detect_features_drift(raw_data.loc[raw_data.dteday.between(reference_dates[0], reference_dates[1])], 
                           raw_data.loc[raw_data.dteday.between(date[0], date[1])],
                           column_mapping=data_columns,
                           get_scores=True)
    
    features_historical_drift_pvalues.append([x[1] for x in drifts])
    
features_historical_drift_pvalues_frame = pd.DataFrame(features_historical_drift_pvalues, 
                                                       columns = data_columns.numerical_features)


In [ ]:
fig = go.Figure(data=go.Heatmap(
                   z = features_historical_drift_pvalues_frame.transpose(),
                   x = [x[1] for x in experiment_batches],
                   y = features_historical_drift_pvalues_frame.columns,
                   hoverongaps = False,
                   xgap = 1,
                   ygap = 1,
                   zmin = 0,
                   zmax = 1,
                   colorscale = 'reds_r'
                   )
               )

fig.update_xaxes(side="top")

fig.update_layout(
    xaxis_title = "Timestamp",
    yaxis_title = "p-value"
)

fig.show("notebook")

## Dataset Drift

In [ ]:
dataset_historical_drift = []

for date in experiment_batches:
    dataset_historical_drift.append(detect_dataset_drift(raw_data.loc[raw_data.dteday.between(reference_dates[0], reference_dates[1])], 
                           raw_data.loc[raw_data.dteday.between(date[0], date[1])], 
                           column_mapping=data_columns))

In [ ]:
fig = go.Figure(data=go.Heatmap(
                   z = [[1 if x == True else 0 for x in dataset_historical_drift]],
                   x = [x[1] for x in experiment_batches],
                   y = [''],
                   hoverongaps = False,
                   xgap = 1,
                   ygap = 1,
                   zmin = 0,
                   zmax = 1,
                   colorscale = 'Bluered',
                   showscale = False
                   )
               )

fig.update_xaxes(side="top")

fig.update_layout(
    xaxis_title = "Timestamp",
    yaxis_title = "Dataset Drift"
)
fig.show()

In [ ]:
dataset_historical_drift_ratio = []

for date in experiment_batches:
    dataset_historical_drift_ratio.append(detect_dataset_drift(raw_data.loc[raw_data.dteday.between(reference_dates[0], reference_dates[1])], 
                           raw_data.loc[raw_data.dteday.between(date[0],date[1])],
                           column_mapping=data_columns,
                           get_ratio=True))

In [ ]:
fig = go.Figure(data=go.Heatmap(
                   z = [dataset_historical_drift_ratio],
                   x = [x[1] for x in experiment_batches],
                   y = [''],
                   hoverongaps = False,
                   xgap = 1,
                   ygap = 1,
                   zmin = 0.5,
                   zmax = 1,
                   colorscale = 'reds'
                  )
               )

fig.update_xaxes(side="top")

fig.update_layout(
    xaxis_title = "Timestamp",
    yaxis_title = "Dataset Drift"
)
fig.show()

## Log Dataset Drift in MLFlow

In [ ]:
#log into MLflow
client = MlflowClient()

#set experiment
mlflow.set_experiment('Dataset Drift Analysis with Evidently')

#start new run
for date in experiment_batches:
    with mlflow.start_run() as run: 
        
        # Log parameters
        mlflow.log_param("begin", date[0])
        mlflow.log_param("end", date[1])

        # Log metrics
        metric = detect_dataset_drift(raw_data.loc[raw_data.dteday.between(reference_dates[0], reference_dates[1])], 
                           raw_data.loc[raw_data.dteday.between(date[0], date[1])],
                           column_mapping=data_columns,
                           get_ratio=True)
        
        mlflow.log_metric('dataset drift', metric)

        print(run.info)

In [ ]:
#run MLflow UI (it will be more convinient to run it directly from the terminal)
#!mlflow ui

# Support Evidently
Enjoyed the tutorial? Star Evidently on GitHub to contribute back! This helps us continue creating free open-source tools for the community. https://github.com/evidentlyai/evidently